## Imports

In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import catboost as cb

## Processing

In [3]:
df = (pd
    .read_csv("./data/superset.csv")
    .set_axis([df["Unnamed: 0"]]) 
    .drop(["Unnamed: 0"], axis=1)
)

In [6]:
df.Subgroup = df.Subgroup.replace(
    {cat:num for num, cat in enumerate(df.Subgroup.unique())}
    ) # Create numerical target (easier for the models to use)

## Splitting

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_ = df.drop(columns=["Subgroup"])
y_ = df.Subgroup

# Initial Datasets
X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.25, random_state=42)

## Building

### CatBoost

https://catboost.ai/docs/concepts/about.html

In [10]:
print(cb.__version__) # Consistency

0.25.1


In [13]:
X_trainset, X_val, y_trainset, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Apply a validation-like training set
train_set = cb.Pool(X_trainset, label=y_trainset)
eval_set = cb.Pool(X_val, label=y_val)

entire_train = cb.Pool(X_train, label=y_train)

In [16]:
# Set grid search params
cbc_params = {
    'random_seed': 88,
    'verbose': 10,
    'loss_function': 'MultiClass'}

CBC = cb.CatBoostClassifier(**cbc_params)

grid_params ={
    'iterations': [100,500,1000],
    'learning_rate': [0.01,0.05,0.1],
    'l2_leaf_reg': [1,3,5,7,9,11,20],
    'depth': [6, 8, 10],
    'random_strength': [0.01, 0.05, 0.1]}

# Grid Search does automatic CV. So we only need to parse the initial training data
CBC_grid = CBC.grid_search(grid_params, X=entire_train, partition_random_seed=12)

0:	learn: 1.0885020	test: 1.0958680	best: 1.0958680 (0)	total: 196ms	remaining: 19.4s
10:	learn: 0.9826027	test: 1.0503633	best: 1.0503633 (10)	total: 1.23s	remaining: 9.95s
20:	learn: 0.8891768	test: 1.0058475	best: 1.0058475 (20)	total: 2.27s	remaining: 8.54s
30:	learn: 0.8111748	test: 0.9704574	best: 0.9704574 (30)	total: 3.32s	remaining: 7.4s
40:	learn: 0.7443546	test: 0.9424778	best: 0.9424778 (40)	total: 4.25s	remaining: 6.11s
50:	learn: 0.6841978	test: 0.9128077	best: 0.9128077 (50)	total: 5.18s	remaining: 4.97s
60:	learn: 0.6315727	test: 0.8877014	best: 0.8877014 (60)	total: 6.13s	remaining: 3.92s
70:	learn: 0.5836329	test: 0.8643433	best: 0.8643433 (70)	total: 7.15s	remaining: 2.92s
80:	learn: 0.5392344	test: 0.8478009	best: 0.8478009 (80)	total: 8.16s	remaining: 1.91s
90:	learn: 0.5003690	test: 0.8321426	best: 0.8320123 (89)	total: 8.97s	remaining: 887ms
99:	learn: 0.4689253	test: 0.8176843	best: 0.8176843 (99)	total: 9.71s	remaining: 0us

bestTest = 0.8176843356
bestIteratio

In [ ]:
CBC_grid.params

,Feature Id,Importances
0,2184,28.928688
1,672,1.648304
2,1687,1.583237
3,833,1.455189
4,1656,1.437726
...,...,...
2829,2826,0.000000
2830,2827,0.000000
2831,2828,0.000000
2832,2830,0.000000


In [ ]:
FCBC = cb.CatBoostClassifier(**CBC_grid.params)
FCBC.fit(train_set)

In [ ]:
final_y_preds = FCBC.predict(eval_set)
final_y_preds = np.reshape(y_preds, y_val.values.shape)

In [ ]:
f"{round(100 - len(*np.nonzero(y_preds - y_val.values)) / len(y_preds) * 100)}% Accuracy"

'79% Accuracy'

In [ ]:
# my_best_model.save_model('catboost_model.bin')

### Default CatBoost Test

In [ ]:
default_CBC = cb.CatBoostClassifier()

In [ ]:
default_CBC.fit(
    train_set,
    eval_set=(X_val, y_val),
    verbose=False,
)

KeyboardInterrupt: 

In [ ]:
def_y_preds = default_CBC.predict(eval_set)
def_y_preds = np.reshape(def_y_preds, y_val.values.shape)

In [ ]:
f"{round(100 - len(*np.nonzero(y_preds - y_val.values)) / len(y_preds) * 100)}% Accuracy"

'79% Accuracy'

### Default CatBoost Decomp

In [ ]:
y_trainset.values.shape

(56,)

In [ ]:
y_val.values.shape

(19,)

In [ ]:
avg_expected_loss_CBC, avg_bias_CBC, avg_var_CBC = bias_variance_decomp(
        default_CBC, X_trainset.values, y_trainset.values, X_val, y_val, # Used .values since mlxtend extracts better from numpy arrays than pandas dfs
        loss='0-1_loss',
        random_seed=66)

print(f'Average expected loss: {avg_expected_loss_CBC}')
print(f'Average bias: {avg_bias_CBC}')
print(f'Average variance: {avg_var_CBC}')

1:	learn: 1.0426174	total: 151ms	remaining: 1m 15s
2:	learn: 1.0160193	total: 230ms	remaining: 1m 16s
3:	learn: 1.0011996	total: 294ms	remaining: 1m 13s
4:	learn: 0.9774458	total: 381ms	remaining: 1m 15s
5:	learn: 0.9461042	total: 456ms	remaining: 1m 15s
6:	learn: 0.9244746	total: 529ms	remaining: 1m 15s
7:	learn: 0.9097368	total: 600ms	remaining: 1m 14s
8:	learn: 0.8876313	total: 676ms	remaining: 1m 14s
9:	learn: 0.8647059	total: 750ms	remaining: 1m 14s
10:	learn: 0.8488065	total: 829ms	remaining: 1m 14s
11:	learn: 0.8320039	total: 899ms	remaining: 1m 13s
12:	learn: 0.8142801	total: 981ms	remaining: 1m 14s
13:	learn: 0.7965193	total: 1.06s	remaining: 1m 14s
14:	learn: 0.7805144	total: 1.13s	remaining: 1m 14s
15:	learn: 0.7692651	total: 1.22s	remaining: 1m 14s
16:	learn: 0.7545986	total: 1.28s	remaining: 1m 13s
17:	learn: 0.7379340	total: 1.35s	remaining: 1m 13s
18:	learn: 0.7162706	total: 1.43s	remaining: 1m 14s
19:	learn: 0.7063683	total: 1.51s	remaining: 1m 14s
20:	learn: 0.6923661	

ValueError: could not broadcast input array from shape (19,1) into shape (19)

### SKlearn Based GBC

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
hard_params = {'n_estimators': 1000, 'max_leaf_nodes': 4, 'max_depth': None, 'random_state': 2,
                'min_samples_split': 5}

# intermediate params

params = dict(hard_params)
# params.update(inter_params)

In [ ]:
# X_trainset, X_val, y_trainset, y_val
GBC = GradientBoostingClassifier(**params)
GBC.fit(X_trainset, y_trainset)

GradientBoostingClassifier(max_depth=None, max_leaf_nodes=4,
                           min_samples_split=5, n_estimators=1000,
                           random_state=2)

In [ ]:
GBC.score(X_val, y_val)

0.8421052631578947

In [ ]:
# WIP
# attempt to plot the training process(https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html)
# training vs validation deviance


y_val_list = y_val.to_numpy()



test_score = np.zeros((params['n_estimators'],), dtype=np.float64)
for i, y_pred in enumerate(GBC.staged_predict(X_val)):
    test_score[i] = GBC.loss_(y_val_list, y_pred)

fig = plt.figure(figsize=(6, 6))
plt.subplot(1, 1, 1)
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, reg.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')
fig.tight_layout()
plt.show()


ValueError: operands could not be broadcast together with shapes (19,3) (19,) 

### SKlearn decomposition

In [ ]:
from mlxtend.evaluate import bias_variance_decomp

In [ ]:
def_GBC = GradientBoostingClassifier(**params)

avg_expected_loss_GBC, avg_bias_GBC, avg_var_GBC = bias_variance_decomp(
        def_GBC, X_trainset.values, y_trainset.values, X_val, y_val, # Used .values since mlxtend extracts better from numpy arrays than pandas dfs
        loss='0-1_loss',
        random_seed=56)

print(f'Average expected loss: {avg_expected_loss_GBC}')
print(f'Average bias: {avg_bias_GBC}')
print(f'Average variance: {avg_var_GBC}')

Average expected loss: 0.22289473684210523
Average bias: 0.15789473684210525
Average variance: 0.16210526315789475


## k-fold cross-validation:
It seems like you can implement the cross-validation for the train-validation split in this way. (Use to evaluate different models using k-fold CV on the training set
 & pick the final model, which is then used to predict the test set!)

 *Question*: 

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

model = GBC
kfold = KFold(n_splits=7,shuffle = True, random_state=10)
results = cross_val_score(model, X_trainset, y_trainset, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 76.79% (12.37%)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c915e4f9-60c2-40b5-a522-8a90cb3fd50a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>